In [1]:
from sklearn. discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.linear_model import LogisticRegression
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn import datasets, svm
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")
from ucimlrepo import fetch_ucirepo 


In [2]:
mushroom = fetch_ucirepo(id=73) 
  
X = mushroom.data.features 
y = mushroom.data.targets 

df_mushroom = pd.DataFrame(X.join(y))
df_mushroom.rename(columns = {'poisonous':'target'}, inplace = True )
df_mushroom


,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,...,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,...,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [3]:
labelencoder=LabelEncoder()
for column in df_mushroom.columns:
    df_mushroom[column] = labelencoder.fit_transform(df_mushroom[column])
    
df_mushroom = df_mushroom.join(y)
df_mushroom.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,target,poisonous
0,5,2,4,1,6,1,0,1,4,0,...,7,0,2,1,4,2,3,5,1,p
1,5,2,9,1,0,1,0,0,4,0,...,7,0,2,1,4,3,2,1,0,e
2,0,2,8,1,3,1,0,0,5,0,...,7,0,2,1,4,3,2,3,0,e
3,5,3,8,1,6,1,0,1,5,0,...,7,0,2,1,4,2,3,5,1,p
4,5,2,3,0,5,1,1,0,4,1,...,7,0,2,1,0,3,0,1,0,e


In [4]:
df_mushroom_for_classes = df_mushroom.drop('target', axis = 1).groupby(['poisonous'])
df_mushroom_for_classes
# sns.pairplot(df_mushroom[['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat', 'poisonous']], palette= sns.color_palette("RdPu", 3),  hue = 'poisonous')